In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import gdal
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Set max columns and rows displayed
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows', 10000)

In [3]:
# Read in csv files using pandas
accessolar_data = pd.read_csv("./data/ACCESSolar/NYCHA_ACCESSolar_Opportunities.csv")
utility_data = pd.read_csv("./data/Typical_Utility_Bill_Information_Electric__Beginning_2011.csv")

In [4]:
# Checking the ACCESSolar data to see if it came in right. Will need some cleaning but the read looks solid. 
accessolar_data.head()

,DEVELOPMENT,STREET ADDRESS,POSTCODE,BOROUGH,BLDG. NUMBER,TDS NUMBER,BIN,No. of FLOORS,ROOF SPACE (Sq. Ft.),ESTIMATED ROOF SOLAR CAPACITY (kW),ADJUSTED VALUE (kW),ROOF CONDITION RATING OR REPLACEMENT DATE,No. of RESIDENTIAL UNITS,No. of TOTAL UNITS,SENIOR DEVELOPMENT (YES/NO),Latitude,Longitude,Community Board,Council District,Census Tract,BBL,NTA,Location 1
0,POLO GROUNDS TOWERS,3005 FREDERICK DOUGLASS BOULEVARD,10039,MANHATTAN,8,149,1810098,2.0,19804.34,67.25,47.07,3,0.0,0,Non-residential,NaN,NaN,10,9,24302,1021060003,Central Harlem North-Polo Grounds,NaN
1,ROBBINS PLAZA,341 EAST 70TH STREET,10021,MANHATTAN,1,218,1044841,20.0,8774.21,29.32,29.32,5,150.0,150,Yes,40.766730,-73.957495,8,5,126,1014450023,Lenox Hill-Roosevelt Island ...,"(40.76673, -73.957495)"
2,ISAACS,1806 1ST AVENUE,10128,MANHATTAN,3,139,1082365,24.0,7942.03,33.06,33.06,3,206.0,207,No,40.781684,-73.945848,8,5,152,1015730001,Yorkville ...,"(40.781684, -73.945848)"
3,ADAMS,815 EAST 152ND STREET,10455,BRONX,7,118,2091989,21.0,6279.41,43.92,30.74,2,143.0,143,No,40.815451,-73.905192,1,17,79,2026650001,Melrose South-Mott Haven North ...,"(40.815451, -73.905192)"
4,HIGHBRIDGE GARDENS,1135 UNIVERSITY AVENUE,10452,BRONX,4,78,2095218,14.0,7112.23,40.67,28.47,2,117.0,118,No,40.837014,-73.928344,4,16,193,2025270032,Highbridge ...,"(40.837014, -73.928344)"


In [5]:
# Same check but on utility billing data. This looks bad - potentially a flawed convert from a formatted excel sheet?
# Going to check the source to see if the data is available in other formats. 
# UPDATE: seems like the source data is just... like this. Strange that a dataset this small would be this poorly formatted. 
# On the plus side, I traced the data back to the original source (NY Dept of Public Services) which has better-formatted and more recent data. 
utility_data.head()

,Effective Date,Season,Service Type,Company,Customer Type,Customer Details,Demand,Load Factor,Usage,Units of Usage,Total Bill
0,01-Jul-11,SUMMER 2011,ELECTRIC,0,RESIDENTIAL,-,-,-,0,KWH,22.45
1,01-Jul-11,SUMMER 2011,ELECTRIC,0,0,-,-,-,250,0,50.45
2,01-Jul-11,SUMMER 2011,ELECTRIC,0,0,-,-,-,500,0,78.45
3,01-Jul-11,SUMMER 2011,ELECTRIC,0,0,-,-,-,750,0,106.45
4,01-Jul-11,SUMMER 2011,ELECTRIC,0,0,-,-,-,1500,0,190.46


In [6]:
# Overwriting previous utility data with new data adapted from original data source
utility_data = pd.read_csv("./data/Typical Bill- Elecric-Residential- Winter 2013- 8-21-13.csv")

In [7]:
# Checking the read in for THIS version of the data
# Needs some cleaning, but the read looks good! 
utility_data.head()

,Company,Charge Type,0 kWh,250 kWh,500 kWh,750 kWh,1500 kWh,3000 kWh,5000 kWh
0,CENTRAL HUDSON GAS & ELECTRIC CORPORATION,BASIC SERVICE CHARGE,$24.00,$24.00,$24.00,$24.00,$24.00,$24.00,$24.00
1,CENTRAL HUDSON GAS & ELECTRIC CORPORATION,DELIVERY CHARGE,$ -,$12.41,$24.82,$37.22,$74.45,$148.89,$248.15
2,CENTRAL HUDSON GAS & ELECTRIC CORPORATION,BILL CREDIT,$ -,$ -,$ -,$ -,$ -,$ -,$ -
3,CENTRAL HUDSON GAS & ELECTRIC CORPORATION,TEMPORARY STATE ASSESSMENT SURCHGE,$ -,$0.83,$1.67,$2.50,$5.00,$9.99,$16.65
4,CENTRAL HUDSON GAS & ELECTRIC CORPORATION,REVENUE DECOUPLING MECHANISM,$ -,$0.40,$0.81,$1.21,$2.42,$4.83,$8.05


In [8]:
# Checking if I can read in NSRDB shape files using geopandas. It works, both for the .dbf and .shp format.
# Unsure if we'll be using this - current plan is to interact directly with the PVWatts calculator - but it's good to know we can work with the files. 
geo_test = gpd.read_file("./data/NSRDB/nsrdb_v3_0_1_1998_2016_dni/nsrdb_v3_0_1_1998_2016_dni.shp")
geo_test.head()

,dni,gid,geometry
0,4.944,1,"POLYGON ((-56.76000 3.95410, -56.72000 3.95410..."
1,5.016,2,"POLYGON ((-56.76000 0.11410, -56.72000 0.11410..."
2,5.112,3,"POLYGON ((-56.48000 -15.36590, -56.44000 -15.3..."
3,5.040,4,"POLYGON ((-56.44000 4.87410, -56.40000 4.87410..."
4,4.296,5,"POLYGON ((-56.36000 -2.28590, -56.32000 -2.285..."


In [9]:
# Attempting to read in one of the rasters using GDAL
raster_test = gdal.Open("./data/Air_Pollution/AnnAvg1_10_300mRaster/aa1_bc300m/dblbnd.adf")

In [10]:
# It's a beautiful thing
type(raster_test)

osgeo.gdal.Dataset

In [11]:
# We can definitely access the rasters and their properties. I'm not sure exactly how to use them in conjunction with the data read into pandas
print("Driver: {}/{}".format(raster_test.GetDriver().ShortName,
                            raster_test.GetDriver().LongName))
print("Size is {} x {} x {}".format(raster_test.RasterXSize,
                                    raster_test.RasterYSize,
                                    raster_test.RasterCount))
print("Projection is {}".format(raster_test.GetProjection()))
geotransform = raster_test.GetGeoTransform()
if geotransform:
    print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
    print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))

Driver: AIG/Arc/Info Binary Grid
Size is 157 x 156 x 1
Projection is PROJCS["NAD83 / New York Long Island",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",41.03333333333333],PARAMETER["standard_parallel_2",40.66666666666666],PARAMETER["latitude_of_origin",40.16666666666666],PARAMETER["central_meridian",-74],PARAMETER["false_easting",984250],PARAMETER["false_northing",0],UNIT["Foot_US",0.3048006096012192],AXIS["X",EAST],AXIS["Y",NORTH]]
Origin = (912815.3055, 273282.0654)
Pixel Size = (984.0, -984.0)


In [12]:
# Let's see what we can get out of these binary files in the shadow data. 
# If the format is difficult to work with, it will probably be better to reverse-engineer the lookup script they included with the download. 
with open("./data/Shadows/slippy-nyc-dec-21/10/300/385.bin", mode='rb') as file:
    fileContent = file.read()

In [13]:
# Let's look at the content
# Commenting this out to save screen real estate, but hey, the data made it! The data and metadata are pretty bare-bones though, so while it would theoretically
# be possible to construct our own lookup based on the slippy hierarchy, it's definitely going to be MUCH more time-efficient to reverse-engineer theirs. 
#fileContent

In [14]:
# Okay, now the actual cleaning and EDA
accessolar_data.shape

(325, 23)

In [15]:
# Okay, wow, that's pretty small. We may need a larger dataset for this. off to get that now...

In [16]:
# Got building footprints shapefile, reading that in with Geopandas
geo_test = gpd.read_file("./data/Building_Footprints/geo_export_8d7509d4-c875-49c9-b71c-f79e6148cd9f.dbf")
geo_test.head()

,base_bbl,bin,cnstrct_yr,doitt_id,feat_code,geomsource,groundelev,heightroof,date_lstmo,time_lstmo,lststatype,mpluto_bbl,name,shape_area,shape_len,geometry
0,3044520815,3394646.0,2009.0,1212853.0,2100.0,Photogramm,18.0,21.608508,2017-08-22,00:00:00.000,Constructed,3044520815,None,854.662433,125.079796,"POLYGON ((-73.87130 40.65717, -73.87136 40.657..."
1,4030640041,4548330.0,1930.0,1226227.0,5110.0,Photogramm,122.0,10.360000,2017-08-17,00:00:00.000,Constructed,4030640041,None,217.594243,60.225858,"POLYGON ((-73.87671 40.71425, -73.87677 40.714..."
2,4139430001,4460479.0,1960.0,581946.0,2100.0,Photogramm,10.0,29.811570,2017-08-22,00:00:00.000,Constructed,4139430001,None,946.427476,123.141941,"POLYGON ((-73.85195 40.66235, -73.85195 40.662..."
3,3049720006,3355684.0,1920.0,858061.0,5110.0,Photogramm,32.0,11.200000,2017-08-17,00:00:00.000,Constructed,3049720006,None,248.678169,63.940817,"POLYGON ((-73.94029 40.64108, -73.94034 40.641..."
4,3055100055,3131737.0,1915.0,568078.0,2100.0,Photogramm,44.0,24.980000,2017-08-22,00:00:00.000,Constructed,3055100055,None,1163.227669,165.608763,"POLYGON ((-73.98999 40.62384, -73.98998 40.623..."


In [17]:
# Got building footprints shapefile, reading that in with Geopandas
geo_test = gpd.read_file("./data/Building_Footprints/geo_export_b656bacd-288f-4b70-a9d7-ff8022520d88.dbf")
geo_test.head()

,base_bbl,bin,cnstrct_yr,doitt_id,feat_code,geomsource,groundelev,heightroof,date_lstmo,time_lstmo,lststatype,mpluto_bbl,name,geometry
0,3044520815,3394646.0,2009.0,1212853.0,2100.0,Photogramm,18.0,21.608508,2017-08-22,00:00:00.000,Constructed,3044520815,None,POINT (-73.87136 40.65721)
1,4030640041,4548330.0,1930.0,1226227.0,5110.0,Photogramm,122.0,10.360000,2017-08-17,00:00:00.000,Constructed,4030640041,None,POINT (-73.87674 40.71427)
2,4139430001,4460479.0,1960.0,581946.0,2100.0,Photogramm,10.0,29.811570,2017-08-22,00:00:00.000,Constructed,4139430001,None,POINT (-73.85201 40.66233)
3,3049720006,3355684.0,1920.0,858061.0,5110.0,Photogramm,32.0,11.200000,2017-08-17,00:00:00.000,Constructed,3049720006,None,POINT (-73.94032 40.64111)
4,3055100055,3131737.0,1915.0,568078.0,2100.0,Photogramm,44.0,24.980000,2017-08-22,00:00:00.000,Constructed,3055100055,None,POINT (-73.98990 40.62388)
